In [62]:
from Functions import *
from Optimizations import *
from Saving import *
import numpy as np
from scipy import stats
from Kernels import *
from utils import *

import plotly.graph_objects as go

from scipy import integrate
import time
import pandas as pd

In [46]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
def C_naught(U, sig, dim):
    def f(*args):
        return np.exp(-U(np.array(args))/sig**2)
    return integrate.dblquad(f, -np.inf, np.inf, -np.inf, np.inf)

In [4]:
C_naught(U, 1, 2)

NameError: name 'U' is not defined

In [ ]:
def I(n):
    return integrate.dblquad(lambda t, x: np.exp(-x*t)/t**n, 0, np.inf, lambda x: 1, lambda x: np.inf)

# Ackely Problem
Bounding the problem by radius 32 and setting the normal derivative to be towards the center

In [3]:
def hyper_cube_enforcer(x, upper_bound=32.768, lower_bound=-32.768):
    filter_lower = x < lower_bound 
    x[filter_lower] = lower_bound
    
    filter_upper = x > upper_bound
    x[filter_upper] = upper_bound
    return x

In [4]:
# U = lambda x: AckleyProblem(x) if (np.linalg.norm(x, axis=0) < np.sqrt(len(x)*32)) else float("inf")
# grad_U = lambda x: GradAckleyProblem(x) if (np.linalg.norm(x, axis=0) < np.sqrt(len(x)*32**2)) else x/np.linalg.norm(x, axis=0)

U = lambda x: np.linalg.norm(x, axis=0)**2 #AckleyProblem
grad_U = lambda x: 2*x#GradAckleyProblem


In [6]:
X = np.linspace(-20, 20, 200)
Y = np.linspace(-20, 20, 200)

inp = np.array(np.meshgrid(X, Y))
out = U(inp)


fig = go.Figure(data=[go.Surface(z=out, x=X, y=Y)])

fig.update_layout(title='Ackely Function', autosize=False,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

We want to find for each process the time it take to get into a $$\rho$$ neighborhood of the optimal (in this case (0,0)). Constrain the problem such that in two consecutive steps the process is within the neighborhood (i.e. the distance between the two points is less than $$\rho * 2$$. 

In [37]:
def constant_temperature(t):
    return 5

In [38]:
rho = 1e-5 #0.118

In [39]:
start_position = np.array([20, 20])

In [40]:
def gamma(t):
    return 0.5 / np.log(t + 2) #4/((t // 20) + 1)

def temperature(t):
    return 1 / np.log(t + 2)
#     return 4/((t // 20) + 1)

### Standard Simulated Annealing

In [56]:
ssa_gamma = gamma
ssa_temperature = constant_temperature #temperature
num_particles = 1
end_t = 10000
epsilon = 2 * rho

In [57]:
num_trials = 20

analytics = {
    "time": [],
    "total_iterations": [],
    "end_point": []
}

paths = []

for i in range(num_trials):
    s_time = time.time()
    p_path = simulated_annealing_janky(U, grad_U, start_position, epsilon, ssa_gamma, ssa_temperature,
                                                 start_t=0, end_t=end_t, domain_enforcer=hyper_cube_enforcer)
    paths.append(p_path)
    analytics["time"].append(time.time() - s_time)
    analytics["total_iterations"].append(len(p_path))
    analytics["end_point"].append(p_path[-1])
    
    

In [64]:
analytics_sa = pd.DataFrame(analytics)

### Standard + Constant Simulated Annealing

In [ ]:
standard_particles = 10
standard_paths = []

for i in range(standard_particles):
    path = simulated_annealing_janky(U, grad_U, start_position, rho*2, gamma, temperature,
                                             start_t=0, end_t=20000)
    standard_paths.append(path)
    
    
const_paths = []
const_particles = 10

for i in range(const_particles):
    path = simulated_annealing_janky(U, grad_U, start_position, rho*2, gamma, constant_temperature,
                                             start_t=0, end_t=20000)
    const_paths.append(path)

### Interactive Diffusion (Approximate first process)

In [65]:
k = 1
total_iter = 10000

# first proccess
first_process = {
    "start": np.array([start_position for _ in range(100)]),
    "gamma": gamma,
    "temperature": constant_temperature,
    "num_iter": 10,
    "epsilon": 0
}

# second proccess
second_process = {
    "start":  np.array([start_position for _ in range(1)]),
    "gamma": gamma,
    "temperature": temperature,
    "num_iter": 1,
    "epsilon": 2 * rho
}

In [ ]:
num_trials = 20

analytics = {
    "time": [],
    "total_iterations": [],
    "main_process_iter": [],
    "end_point": []
}

paths = []

for i in range(num_trials):
    s_time = time.time()
    p_path, two_p_path = interactive_diffusion(U, grad_U, first_process, second_process, total_iter, k=1/2, verbose=False)  
    analytics["time"].append(time.time() - s_time)
    analytics["main_process_iter"].append(sum([len(p) for p in two_p_path]))
    analytics["total_iterations"].append(sum([len(p) for p in p_path]) + sum([len(p) for p in two_p_path]))
    analytics["end_point"].append(two_p_path[-1][-1])
    
    paths.append(p_path)

    
    

In [ ]:
analytics_ia_aprox = pd.DataFrame(analytics)

### Interactive Diffusion (Assume Gibbs Distribution for first)

In [ ]:
dim = 10
k = 1
total_iter = 500
sig = 1

# proccess
process = {
    "start":  np.array([start_position for _ in range(1)]),
    "gamma": gamma,
    "temperature": temperature,
    "num_iter": 1,
    "epsilon": 2 * rho
}

In [ ]:
num_trials = 5

analytics = {
    "time": [],
    "total_iterations": [],
    "main_process_iter": [],
    "end_point": []
}

paths = []

for i in range(num_trials):
    s_time = time.time()
    p_path = interactive_diffusion_gibbs(U, grad_U, process, total_iter, k, sig, domain_enforcer=hyper_cube_enforcer)
    analytics["time"].append(time.time() - s_time)
    analytics["total_iterations"].append(sum([len(p) for p in p_path]))
    analytics["end_point"].append(p_path[-1][-1])
    
    paths.append(p_path)

    

In [ ]:
analytics_ia_gibbs = pd.DataFrame(analytics)